`homework_anki_vocab_20250108.ipynb`에 작성하세요.


# LLM 프롬프트로 Anki 단어장 생성하기

### 실습 목표

* 영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성하는 프롬프트를 설계한다.
* 프롬프트 내에 시스템 지시사항, 예시, 출력 형식을 포함한다.

### 문제

**다음 요구사항을 만족하는 LLM 프롬프트를 작성하시오.**

#### 요구사항

1. **시스템 지시사항**

   * LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
   * 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
   * 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
   * 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

2. **예시**

   * 입력 텍스트 예시:

     ```
     The committee will convene next week to discuss the new policy proposals and reach a consensus.
     ```
   * 출력 예시:

     ```
     convene;to come together for a meeting;verb;The committee will convene next week.
     consensus;general agreement;noun;The group reached a consensus after a long discussion.
     proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
     ```

3. **LLM 출력 형식**

   * 각 카드: `단어 또는 어구;뜻;품사;영어 예문`
   * 모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
   * 불필요한 부가설명, 줄바꿈, 문장 없음

4. **DataFrame 시각화**

    * LLM 응답을 파싱해서 다음과 같이 DataFrame으로 출력하세요.

![Image](https://d.pr/i/otRx7u+)



# LLM 프롬프트로 Anki 단어장 생성하기

### 실습 목표

* 영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성하는 프롬프트를 설계한다.
* 프롬프트 내에 시스템 지시사항, 예시, 출력 형식을 포함한다.

In [ ]:
# 필요한 라이브러리 설치
!pip install openai pandas

In [ ]:
# ==========================================
# [Step 1] 라이브러리 설치 (최초 1회 실행)
# ==========================================
!pip install openai pandas

# ==========================================
# [Step 2] 전체 코드 실행
# ==========================================
import os
import pandas as pd
from io import StringIO
from getpass import getpass
from openai import OpenAI

# 1. API 키 안전하게 입력받기
# 코랩에서 실행 시 입력창이 뜹니다. 키를 붙여넣고 엔터를 치세요.
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요 (sk-...): ")

# 2. 클라이언트 초기화
client = OpenAI()

# 3. 시스템 프롬프트 설계 (과제 요구사항 반영)
system_prompt = """
### Role
You are an expert English teacher and Anki card generator.

### Task
Analyze the given English text and extract valuable vocabulary (advanced words, idioms, specific terminology).
Exclude very basic words.

### Output Format
- Format: word_or_phrase;definition;part_of_speech;example_sentence
- Separator: semicolon (;)
- Definition: English or Korean
- Structure: One card per line, no header, no markdown tags.

### Example
Input: "The committee will convene next week to discuss the new policy proposals."
Output:
convene;to come together for a meeting;verb;The committee will convene next week.
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
"""

# 4. 테스트 입력 텍스트
user_input_text = """
The rapid advancement of artificial intelligence has precipitated a paradigm shift in various industries. 
Engineers are striving to mitigate potential biases in algorithms while enhancing their efficacy.
"""

print("⏳ AI가 단어장을 생성하고 있습니다...")

# 5. LLM API 호출
try:
    response = client.chat.completions.create(
        model="gpt-4o",  # gpt-4o 또는 gpt-3.5-turbo 사용 가능
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input_text}
        ],
        temperature=0  # 일관된 결과를 위해 0 설정
    )

    # 6. 응답 처리 및 DataFrame 변환
    anki_result = response.choices[0].message.content.strip()
    
    # 불필요한 마크다운 태그 제거
    anki_result = anki_result.replace("```csv", "").replace("```", "").strip()

    print("--- [LLM 원본 출력] ---")
    print(anki_result)
    print("-" * 30)

    # CSV 형식의 문자열을 DataFrame으로 변환
    df = pd.read_csv(
        StringIO(anki_result), 
        sep=";", 
        names=["단어/어구", "뜻", "품사", "예문"], 
        header=None,
        engine='python'
    )
    
    print("\n--- [Anki 단어장 DataFrame] ---")
    display(df)

except Exception as e:
    print(f"\n❌ 오류가 발생했습니다: {e}")
    print("API Key가 올바른지, 혹은 크레딧(잔액)이 있는지 확인해주세요.")